In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import models
from PIL import Image

In [2]:
!pip install pytorch-pretrained-biggan
!pip install ranger-adabelief



In [3]:
import os
import shutil
import nltk

# Define the expected NLTK directory structure
nltk_data_path = '/kaggle/working/nltk_data/corpora/'
os.makedirs(nltk_data_path, exist_ok=True)

# Path to the uploaded wordnet directory
wordnet_source_path = '/kaggle/input/wordnetn/wordnet/'
wordnet_dest_path = os.path.join(nltk_data_path, 'wordnet')

# Move or copy the wordnet folder to the expected location
if not os.path.exists(wordnet_dest_path):
    shutil.copytree(wordnet_source_path, wordnet_dest_path)

# Add the working directory to NLTK's data path
nltk.data.path.append('/kaggle/working/nltk_data/')
# Test WordNet
from nltk.corpus import wordnet

try:
    syns = wordnet.synsets('example')
    print(f"WordNet loaded successfully. Example synsets: {syns}")
except Exception as e:
    print(f"Error loading WordNet: {e}")

WordNet loaded successfully. Example synsets: [Synset('example.n.01'), Synset('model.n.07'), Synset('exemplar.n.01'), Synset('example.n.04'), Synset('case.n.01'), Synset('exercise.n.04')]


In [4]:
import nltk

nltk.data.path.append('/kaggle/working/nltk_data')
from nltk.corpus import wordnet

print(wordnet.synsets('horse'))

[Synset('horse.n.01'), Synset('horse.n.02'), Synset('cavalry.n.01'), Synset('sawhorse.n.01'), Synset('knight.n.02'), Synset('horse.v.01')]


In [5]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from PIL import Image
import os
from pytorch_pretrained_biggan import BigGAN, one_hot_from_names, truncated_noise_sample

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# BigGAN model
biggan = BigGAN.from_pretrained('biggan-deep-128').to(device)

# File paths
classes_file = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/classes.txt"
predicate_matrix_file = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/predicate-matrix-continuous.txt"
train_dir = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/train"

# Load classes and predicate matrix
def load_classes(file_path):
    with open(file_path, "r") as f:
        classes = [line.strip().split('\t')[-1] for line in f.readlines()]
    return classes

classes = load_classes(classes_file)
folder_classes = set(os.listdir(train_dir))
unseen_classes = [cls for cls in classes if cls not in folder_classes]

# Mapping classes to indices
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
unseen_class_to_idx = {cls: idx + len(folder_classes) for idx, cls in enumerate(unseen_classes)}

# Manual synset mapping for problematic classes
manual_synsets = {
    "moose": "elk.n.01",
    "rhinoceros": "rhinoceros.n.01",
    "squirrel": "squirrel.n.01"
}

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, root_dir, class_to_idx, transform=None):
        self.root_dir = root_dir
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.data = []
        for class_name in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                class_idx = class_to_idx[class_name]
                for img_name in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_name)
                    self.data.append((img_path, class_idx))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):  
        
        img_path, label = self.data[idx]
        # Check if the data is already a PIL Image
        if isinstance(img_path, Image.Image):
            image = img_path
        else:
            # Open the image file
            image = Image.open(img_path).convert('RGB')
    
        if self.transform:
            image = self.transform(image)
        return image, label


# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageDataset(train_dir, class_to_idx, transform)

def generate_images_for_class(class_name, num_images, class_idx):
    generated_images = []
    try:
        synsets = wordnet.synsets(class_name.replace("+", " "))
        if not synsets:
            print(f"No synset found for {class_name}. Skipping.")
            return []
        
        synset_name = manual_synsets.get(class_name) if class_name in manual_synsets else synsets[0].name()

        for batch_start in range(0, num_images, 50):
            batch_size = min(50, num_images - batch_start)
            class_vector = torch.tensor(
                one_hot_from_names([synset_name.split('.')[0]], batch_size=batch_size), dtype=torch.float32
            ).to(device)
            noise_vector = torch.tensor(
                truncated_noise_sample(truncation=0.4, batch_size=batch_size, seed=42), dtype=torch.float32
            ).to(device)

            with torch.no_grad():
                output = biggan(noise_vector, class_vector, truncation=0.4)

            for i in range(output.shape[0]):
                image = transforms.ToPILImage()(output[i].cpu().squeeze())
                generated_images.append((image, class_idx))

    except Exception as e:
        print(f"Error generating images for {class_name}: {e}")

    return generated_images

# Generate synthetic data
print("Generating images...")
new_data = []

for class_name in classes:
    class_idx = class_to_idx.get(class_name, unseen_class_to_idx.get(class_name))
    if class_name in folder_classes:
        num_images = 250
    else:
        num_images = 500

    generated_images = generate_images_for_class(class_name, num_images, class_idx)
    new_data.extend(generated_images)
    print(f"Class {class_name}: {len(generated_images)} images added.")

# Add generated data to the dataset
for img, label in new_data:
    dataset.data.append((img, label))

# Final dataset size
print(f"Total new images added: {len(new_data)}")
print(f"Final dataset size: {len(dataset)}")

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, valid_ds = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=8)
val_loader = DataLoader(valid_ds, batch_size=32, shuffle=False, num_workers=8)

print(f"Training set size: {len(train_ds)}")
print(f"Validation set size: {len(valid_ds)}")

import timm

# Model initialization using Swin Transformer
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=50)
model = model.to(device)

for param in model.parameters():
    param.requires_grad = True


# Define loss, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4) 

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)


# Training and validation loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrics calculation
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Train Loss: {loss.item():.4f}, Train Acc: {100 * correct_train / total_train:.2f}%")

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Metrics calculation
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = 100 * correct_val / total_val

    # Scheduler step
    scheduler.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}] Complete, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

print("Training complete.")


100%|██████████| 630/630 [00:00<00:00, 2297749.15B/s]
/usr/local/lib/python3.10/dist-packages/pytorch_pretrained_biggan/model.py:279: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feat

Generating images...
Class antelope: 250 images added.
No synset found for grizzly+bear. Skipping.
Class grizzly+bear: 0 images added.
No synset found for killer+whale. Skipping.
Class killer+whale: 0 images added.
Class beaver: 250 images added.
Class dalmatian: 250 images added.
No synset found for persian+cat. Skipping.
Class persian+cat: 0 images added.
Class horse: 500 images added.
No synset found for german+shepherd. Skipping.
Class german+shepherd: 0 images added.
No synset found for blue+whale. Skipping.
Class blue+whale: 0 images added.
No synset found for siamese+cat. Skipping.
Class siamese+cat: 0 images added.
Error generating images for skunk: 
Class skunk: 0 images added.
Error generating images for mole: 
Class mole: 0 images added.
Class tiger: 250 images added.
Class hippopotamus: 250 images added.
Class leopard: 250 images added.
Error generating images for moose: 
Class moose: 0 images added.
No synset found for spider+monkey. Skipping.
Class spider+monkey: 0 images

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


model.safetensors:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch [1/5], Batch [100/452], Train Loss: 0.5644, Train Acc: 62.22%
Epoch [1/5], Batch [200/452], Train Loss: 0.4057, Train Acc: 73.41%
Epoch [1/5], Batch [300/452], Train Loss: 0.4757, Train Acc: 77.85%
Epoch [1/5], Batch [400/452], Train Loss: 0.4891, Train Acc: 80.16%
Epoch [1/5] Complete, Train Loss: 0.6971, Train Acc: 80.97%, Val Loss: 0.3030, Val Acc: 90.39%
Epoch [2/5], Batch [100/452], Train Loss: 0.5580, Train Acc: 91.91%
Epoch [2/5], Batch [200/452], Train Loss: 0.1942, Train Acc: 91.67%
Epoch [2/5], Batch [300/452], Train Loss: 0.3113, Train Acc: 91.77%
Epoch [2/5], Batch [400/452], Train Loss: 0.2390, Train Acc: 91.74%
Epoch [2/5] Complete, Train Loss: 0.2545, Train Acc: 91.68%, Val Loss: 0.3317, Val Acc: 89.53%
Epoch [3/5], Batch [100/452], Train Loss: 0.1658, Train Acc: 94.88%
Epoch [3/5], Batch [200/452], Train Loss: 0.1219, Train Acc: 95.12%
Epoch [3/5], Batch [300/452], Train Loss: 0.2835, Train Acc: 95.23%
Epoch [3/5], Batch [400/452], Train Loss: 0.0292, Train Acc: 9

In [6]:
import os
import pandas as pd
from PIL import Image
import torch


# Prediction
test_dir = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/test"
model.eval()
test_images = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
test_predictions = []

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        predicted_class = torch.argmax(outputs, dim=1).item()
        class_name = classes[predicted_class]  # Get the class name
        test_predictions.append((img_name, class_name))

# Save Predictions
submission = pd.DataFrame(test_predictions, columns=['image_id', 'class'])
submission.to_csv("/kaggle/working/submission.csv", index=False)